In [ ]:
# !pip install kaggle tensorflow_io==0.23.1
# !pip install -U tensorboard_plugin_profile



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.1 MB 6.5 MB/s 
     |████████████████████████████████| 2.1 MB 12.9 MB/s 
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.26.0
    Uninstalling tensorflow-io-gcs-filesystem-0.26.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.26.0


In [ ]:
# from google.colab import files
# from os.path import exists

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
  
# # Then move kaggle.json into the folder where the API expects to find it.
# !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 66 bytes


In [ ]:
# !kaggle competitions download -c hubmap-organ-segmentation
# !unzip /content/hubmap-organ-segmentation.zip

100% 5.78G/5.78G [00:42<00:00, 181MB/s]
100% 5.78G/5.78G [00:42<00:00, 145MB/s]
Archive:  /content/hubmap-organ-segmentation.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: test_images/10078.tiff  
  inflating: train.csv               
  inflating: train_annotations/10044.json  
  inflating: train_annotations/10274.json  
  inflating: train_annotations/10392.json  
  inflating: train_annotations/10488.json  
  inflating: train_annotations/10610.json  
  inflating: train_annotations/10611.json  
  inflating: train_annotations/10651.json  
  inflating: train_annotations/10666.json  
  inflating: train_annotations/10703.json  
  inflating: train_annotations/10892.json  
  inflating: train_annotations/10912.json  
  inflating: train_annotations/10971.json  
  inflating: train_annotations/10992.json  
  inflating: train_annotations/11064.json  
  inflating: train_annotations/1123.json  
  inflating: train_annotations/11448.json  
  inflating: tra

In [ ]:

# %load_ext tensorboard
# !mkdir /tmp/tf_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
mkdir: cannot create directory ‘/tmp/tf_logs’: File exists


In [31]:
%tensorflow_version 2.x
%matplotlib inline

# Operation Modules
import os
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
import pandas as pd


# Visualization Tools
from PIL import Image
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
# from tensorboard import notebook
# notebook.display()

In [32]:
import csv
import sys
csv.field_size_limit(sys.maxsize)

9223372036854775807

In [43]:
IMAGE_BATCH_SIZE = 1 # Singular batches of differing-size images allows batches to avoid ragged-tensor behavior.

In [54]:
def simple_sample(ds):
  for batch in ds.take(1):
    for key, value in batch.items():
      # print(f"{key:20s}: {value} @ {value.numpy().shape}:{value.dtype}")
      print(f"{key:20s}: {value} @ {value.dtype}")

In [55]:
hubmap_train_ds = tf.data.experimental.make_csv_dataset("/content/train.csv", batch_size=IMAGE_BATCH_SIZE).unbatch()

### Guarantee testing with unequal image dims
# hubmap_train_ds = hubmap_train_ds.filter(lambda k: tf.math.reduce_any(k["img_width"] != 3000, axis=-1)); print("WARNING!! Data discarded pointlessly. Remove alterations to hubmap_train_ds, unless you are performing pipeline verification for unequal image sizes.")
# hubmap_train_ds = hubmap_train_ds.filter(lambda k: tf.math.reduce_all(k["img_width"] == 3000, axis=-1)); print("WARNING!! Data discarded pointlessly. Remove alterations to hubmap_train_ds, unless you are performing pipeline verification for guaranteed equal image sizes.")

simple_sample(hubmap_train_ds)

id                  : 29296 @ <dtype: 'int32'>
organ               : b'prostate' @ <dtype: 'string'>
data_source         : b'HPA' @ <dtype: 'string'>
img_height          : 2416 @ <dtype: 'int32'>
img_width           : 2416 @ <dtype: 'int32'>
pixel_size          : 0.4000000059604645 @ <dtype: 'float32'>
tissue_thickness    : 4 @ <dtype: 'int32'>
rle                 : b'54751 20 57166 22 59581 24 61996 25 64411 27 66817 48 69232 50 71647 52 74062 53 76477 55 78892 85 81307 87 83722 90 86137 92 88552 103 90967 105 93382 107 95797 109 98212 111 100627 122 103042 124 105457 126 107872 128 110287 130 112702 141 115118 142 117534 144 119950 145 122366 183 124781 185 127196 187 129430 10 129612 187 131841 17 132027 189 133991 10 134256 18 134442 257 136406 12 136626 10 136664 27 136857 259 138821 14 139038 70 139272 261 141236 15 141450 75 141687 262 143651 17 143866 75 144102 264 146056 39 146278 80 146517 267 148470 42 148692 83 148933 269 150883 46 151104 88 151349 272 153297 49 153520 88 1

In [56]:
def stack_width_height(d):
  d["img_w_h"] = tf.stack([d["img_width"], d["img_height"]], axis=-1, name="stack_width_height.stack_width_height")
  d.pop("img_width")
  d.pop("img_height")
  return d

hubmap_train_ds_wh = hubmap_train_ds.map(stack_width_height, name="Stack_Width_Height")

simple_sample(hubmap_train_ds_wh)

id                  : 28657 @ <dtype: 'int32'>
organ               : b'spleen' @ <dtype: 'string'>
data_source         : b'HPA' @ <dtype: 'string'>
pixel_size          : 0.4000000059604645 @ <dtype: 'float32'>
tissue_thickness    : 4 @ <dtype: 'int32'>
rle                 : b'1077928 16 1080686 22 1083445 27 1086197 37 1088957 38 1091717 39 1094477 40 1097230 48 1099988 51 1102746 54 1105504 57 1108262 60 1111019 64 1113777 67 1116534 71 1119291 75 1122049 78 1124806 82 1127564 85 1130323 87 1133083 88 1135842 90 1138601 92 1141359 95 1144118 97 1146877 99 1149635 102 1152393 105 1155151 108 1157910 110 1160668 113 1163427 115 1166179 124 1168938 126 1171698 127 1174457 129 1177216 131 1179975 133 1182735 134 1185494 136 1188254 137 1191013 138 1193773 139 1196532 140 1199291 142 1202050 143 1204809 145 1207568 146 1210328 148 1213088 149 1215848 150 1218608 151 1221368 151 1224128 152 1226888 153 1229647 155 1232407 156 1235166 158 1237925 160 1240685 161 1243445 161 1246205 162 12489

In [57]:
def training_img_id_to_path(image_id):
  return tf.strings.join(["/content/train_images/", tf.strings.as_string(image_id), ".tiff"])
  # return tf.map_fn(lambda id: f"/content/train_images/{tf.strings.as_string(id)}.tiff", image_id)

def assign_filepath(batch):
  batch["fpath"] = training_img_id_to_path(batch["id"])
  return batch

In [62]:
hubmap_train_ds_filepath = hubmap_train_ds_wh.map(assign_filepath, name="Expand_Filepaths")
simple_sample(hubmap_train_ds_filepath)

id                  : 4561 @ <dtype: 'int32'>
organ               : b'largeintestine' @ <dtype: 'string'>
data_source         : b'HPA' @ <dtype: 'string'>
pixel_size          : 0.4000000059604645 @ <dtype: 'float32'>
tissue_thickness    : 4 @ <dtype: 'int32'>
rle                 : b'943284 19 946282 26 949279 35 952272 52 955268 59 958266 63 961261 71 964258 76 967256 80 970252 88 973249 94 976247 97 979246 100 982244 104 985242 107 988241 110 991240 113 994238 116 997237 119 1000235 122 1003233 125 1006233 126 1009232 128 1012231 130 1015230 132 1018229 134 1021228 135 1024228 136 1027227 138 1030226 140 1033225 141 1036224 143 1039223 144 1042222 145 1045220 148 1048219 149 1051218 151 1054218 151 1057217 152 1060216 153 1063216 154 1066214 156 1069214 156 1072213 157 1075212 159 1078211 160 1081210 162 1084209 163 1087208 164 1090207 165 1093207 165 1096207 165 1099207 165 1102206 167 1105206 167 1108205 168 1111205 168 1114205 168 1117204 169 1120204 169 1123204 169 1126203 170 112

In [72]:


@tf.function
def ext_load_image(image_fpath, authoritative_dimensions):
    image = tf.io.read_file(image_fpath, name="ext_load_image.io_read_file")

    # I promise it's OK to discard the alpha channel of the data we're using
    image = tfio.experimental.image.decode_tiff(image, name="ext_load_image.decode_tiff")[:, :, :3]

    # Normalize and convert to float##
    image = tf.cast(image, tf.float32)/255.0
    
    # print("imd:", image)
    # image.set_shape(authoritative_dimensions.numpy())
    tf.reshape(image, authoritative_dimensions)
    # print("imds:", image)
    return image


@tf.function
def rle_decode_tf(mask_rle, shape):
  print("mask_rle:", mask_rle)
  print("shape:", shape)
  # shape = tf.convert_to_tensor(shape, tf.int64)
  size = tf.math.reduce_prod(shape)
  # Split string
  s = tf.strings.split(mask_rle)
  s = tf.strings.to_number(s, tf.int64)
  # Get starts and lengths
  starts = s[::2] - 1
  lens = s[1::2]
  # Make ones to be scattered
  total_ones = tf.reduce_sum(lens)
  # ones = tf.ones([total_ones], tf.uint8)
  ones = tf.ones([total_ones], tf.float32)
  # Make scattering indices
  r = tf.range(total_ones)
  lens_cum = tf.math.cumsum(lens)
  s = tf.searchsorted(lens_cum, r, 'right')
  idx = r + tf.gather(starts - tf.pad(lens_cum[:-1], [(1, 0)]), s)
  # Scatter ones into flattened mask
  mask_flat = tf.scatter_nd(tf.expand_dims(idx, 1), ones, [size])
  # Reshape into mask
  return tf.transpose(tf.reshape(mask_flat, shape))


In [89]:
def load_construct_spatial_data(k):
  image = ext_load_image(k["fpath"], tf.concat([k["img_w_h"], [3]], -1))
  seg   = rle_decode_tf(k["rle"], k["img_w_h"])

  constructed = tf.concat([tf.expand_dims(seg, axis=-1), image[:, :, :3]], axis=-1, name="load_constructed_spatial_data.concat_constructed")
  constructed = tf.reshape(constructed, tf.concat([k["img_w_h"], [4]], -1))
  return constructed


def load_construct_spatial_data_from_dataset(d):
  d["spatial_data"] = load_construct_spatial_data(d)
  return d

# def load_construct_spatial_data_from_dataset(d):
#   d["spatial_data"] = tf.map_fn(
#       lambda k: tf.RaggedTensor.from_tensor(
#           load_construct_spatial_data(k)
#       ), 
#       d,
#       # fn_output_signature=tf.RaggedTensorSpec(shape=[None, None, 4], dtype=tf.uint8, ragged_rank=1))
#       fn_output_signature=tf.RaggedTensorSpec(shape=[None, None, 4], dtype=tf.float32, ragged_rank=1))
#   return d

In [90]:
hubmap_train_ds_image_pretransform = hubmap_train_ds_filepath.map(load_construct_spatial_data_from_dataset, num_parallel_calls=tf.data.AUTOTUNE)

In [91]:
simple_sample(hubmap_train_ds_image_pretransform)

id                  : 24097 @ <dtype: 'int32'>
organ               : b'kidney' @ <dtype: 'string'>
data_source         : b'HPA' @ <dtype: 'string'>
pixel_size          : 0.4000000059604645 @ <dtype: 'float32'>
tissue_thickness    : 4 @ <dtype: 'int32'>
rle                 : b'1514218 28 1517214 32 1520213 36 1523211 40 1526209 44 1529208 46 1532206 48 1535205 51 1538204 54 1541204 56 1544203 58 1547202 61 1550201 63 1553200 66 1556199 68 1559198 72 1562197 74 1565197 77 1568196 79 1571196 80 1574195 83 1577195 84 1580194 89 1583194 89 1586192 93 1589190 97 1592190 98 1595189 101 1598188 104 1601188 106 1604187 110 1607187 110 1610186 114 1613186 116 1616185 127 1619184 129 1622184 130 1625183 134 1628183 136 1631183 137 1634183 139 1637182 141 1640182 142 1643181 143 1646181 145 1649181 148 1652181 148 1655180 150 1658180 150 1661180 152 1664179 154 1667179 155 1670179 156 1673179 157 1676179 158 1679179 159 1682178 161 1685178 162 1688178 164 1691178 165 1694177 166 1697176 168 170017

In [ ]:
import time
def benchmark(dataset, num_samples=100, take_before_profile = 2):
  finished_samples = 0
  finished_batches = 0
  
  ds_iterator = iter(dataset)

  for completed_before_profile in range(take_before_profile):
    print(f"Warming up... {completed_before_profile+1} / {take_before_profile}")
    next(ds_iterator)

  start_time = time.perf_counter()
  with tf.profiler.experimental.Profile('/tmp/tf_logs'):
    while finished_samples < num_samples:
      with tf.profiler.experimental.Trace("TraceContext", step_num=finished_batches, _r=1):
        data = next(ds_iterator)
        finished_batches += 1
        finished_samples += IMAGE_BATCH_SIZE
        print(f"@{time.perf_counter() - start_time:5.1f}: {finished_batches} {finished_samples}/{num_samples}")



In [ ]:
# %tensorboard --logdir '/tmp/tf_logs'

In [ ]:
# benchmark(hubmap_train_ds_image_pretransform)

Warming up... 1 / 2
Warming up... 2 / 2
@  2.6: 1 5/100
@  3.1: 2 10/100
@  5.9: 3 15/100
@  6.1: 4 20/100
@  8.5: 5 25/100
@  9.3: 6 30/100
@ 10.9: 7 35/100
@ 12.3: 8 40/100
@ 14.6: 9 45/100
@ 15.1: 10 50/100
@ 18.2: 11 55/100
@ 18.4: 12 60/100
@ 21.0: 13 65/100
@ 21.4: 14 70/100
@ 23.8: 15 75/100
@ 24.7: 16 80/100
@ 27.1: 17 85/100
@ 27.9: 18 90/100
@ 31.0: 19 95/100
@ 31.4: 20 100/100


In [88]:
hubmap_train_ds_image_pretransform.element_spec

OrderedDict([('id', TensorSpec(shape=(), dtype=tf.int32, name=None)),
             ('organ', TensorSpec(shape=(), dtype=tf.string, name=None)),
             ('data_source', TensorSpec(shape=(), dtype=tf.string, name=None)),
             ('pixel_size', TensorSpec(shape=(), dtype=tf.float32, name=None)),
             ('tissue_thickness',
              TensorSpec(shape=(), dtype=tf.int32, name=None)),
             ('rle', TensorSpec(shape=(), dtype=tf.string, name=None)),
             ('age', TensorSpec(shape=(), dtype=tf.float32, name=None)),
             ('sex', TensorSpec(shape=(), dtype=tf.string, name=None)),
             ('img_w_h', TensorSpec(shape=(2,), dtype=tf.int32, name=None)),
             ('fpath', TensorSpec(shape=(), dtype=tf.string, name=None)),
             ('spatial_data',
              TensorSpec(shape=(None, None, 4), dtype=tf.float32, name=None))])